### Dataset description

This dataset contains information about Sales Values in Dollars on American Stores between 2010 and 2011. In particular, this dataset contains:

Stores' Area, State, Region and Size;
Products' ID, Description, Type, Category and Sale Date;
Accounting Info, such as Budget Margin, Profit, Total Expenses and Marking.

The data was extracted between January 1th in 2010 and December 31th in 2011.

### Variables description

* Area Code: Store's Code;
* State: Store's State;
* Market: Store's Region;
* Market Size: Store's Size;
* Profit: Profits in Dollars `($)`;
* Margin: Profit + Total Expenses `($)` OR Sales - COGS `($)`;
* Sales: Values Acquired in Sales `($)`;
* COGS: Cost of Goods Sold `($)`;
* Total Expenses: Total Expenses to get the Product to Sell `($)`;
* Marketing: Expenses in Marketing `($)`;
* Inventory: Inventory Value of the Product in the Sale Moment `($)`;
* Budget Profit: Expected Profit `($)`;
* Budget COGS: Expected COGS `($)`;
* Budget Margin: Expected Profit + Expected Total Expenses `($)` OR Expected Sales - Expected COGS `($)`;
* Budget Sales: Expected Value Acquired in Sales `($)`;
* ProductID: Product ID;
* Date: Sale Date;
* Product Type: Product Category;
* Product: Product Description;
* Type: Type;

### Imports

In [103]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from fast_ml.model_development import train_valid_test_split
import itertools as it
from statsmodels.stats.outliers_influence import variance_inflation_factor


### Load the data

In [60]:
df = pd.read_csv("sales.csv")